Dado que estamos trabajando en un problema de regresión, necesitamos definir una variable objetivo numérica. Supongamos que elegimos Work_Experience como variable objetivo, para predecir la experiencia laboral

### Pasos para el pipeline

1. Preprocesamiento:
Eliminar columnas irrelevantes: La columna ID no es útil para el modelo.

Codificación de variables categóricas:

Gender, Ever_Married, Graduated, Profession, Spending_Score, Var_1 son categóricas y deben codificarse (usar OneHotEncoder).

2. Escalado de variables numéricas:

Age, Work_Experience, Family_Size deben escalarse (usar StandardScaler o MinMaxScaler)
Modelo de regresión:

Usar GradientBoostingRegressor (de sklearn) o XGBRegressor (de XGBoost).

3. Optimización con Grid Search:

Definir un espacio de búsqueda para los hiperparámetros del modelo.

4. Evaluación:

Predecir en el conjunto de prueba (test.csv).

Calcular métricas como MSE y R².

In [13]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

# Cargar datos de entrenamiento y prueba
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# Eliminar filas donde 'Work_Experience' es NaN
train_data = train_data.dropna(subset=['Work_Experience'])
test_data = test_data.dropna(subset=['Work_Experience'])

# Definir características (X) y objetivo (y)
# Supongamos que queremos predecir 'Work_Experience'
X_train = train_data.drop(columns=['ID', 'Work_Experience', 'Segmentation'])  # Eliminar columnas irrelevantes
y_train = train_data['Work_Experience']

X_test = test_data.drop(columns=['ID', 'Work_Experience', 'Segmentation'])  # Eliminar columnas irrelevantes
y_test = test_data['Work_Experience']

# Definir columnas numéricas y categóricas
numeric_features = ['Age', 'Family_Size']
categorical_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']

# Preprocesamiento con imputación
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputar valores faltantes con la mediana
    ('scaler', StandardScaler())  # Escalar características numéricas
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar valores faltantes con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificar características categóricas
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear pipeline con preprocesamiento y modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Definir espacio de búsqueda para Grid Search
param_grid = {
    'regressor__n_estimators': [50, 100, 200],  # Número de árboles
    'regressor__learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'regressor__max_depth': [3, 5, 10],  # Profundidad máxima de los árboles
    'regressor__min_samples_split': [2, 5, 10]  # Mínimo de muestras para dividir un nodo
}

# Configurar Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Entrenar el modelo con Grid Search
grid_search.fit(X_train, y_train)

# Evaluar el mejor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("MSE en test:", mean_squared_error(y_test, y_pred))
print("R² en test:", r2_score(y_test, y_pred))

Mejores hiperparámetros: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 100}
MSE en test: 9.450497113677942
R² en test: 0.15304284162155302
